In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_atendente"
TABELA_DESTINO = "v_credit.gold.dm_atendente"

### Transformacao Silver → Gold (Dimensao Atendente)
Le tb_atendente (Silver) e adiciona coluna derivada de negocio:
- **ds_perfil**: Classificacao descritiva baseada em nu_nivel
  - nu_nivel = 2 → "Especialista (N2)"
  - nu_nivel = 1 → "Generalista (N1)"

Faz MERGE em dm_atendente (Gold) para carregar a dimensao.

**Importante**: Dimensao usada em ft_atendimento_geral e ft_performance_agente.

### Configuracao
Define origem (Silver tb_atendente) e destino (Gold dm_atendente - dimensao do Star Schema).

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_atendente"),
    F.col("nm_atendente"),
    F.col("nu_nivel").cast("smallint"),
    
    F.when(F.col("nu_nivel") == 2, "Especialista (N2)")
    .otherwise("Generalista (N1)").alias("ds_perfil")
)

DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.cd_atendente = s.cd_atendente"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Dimensão Atendente carregada!")